<hr style="height:1px; width:100%">

<h1 style="color:#336699">TASK 05D: Working with Pandas DataFrames - Part 2</h1>

<p style="color:#204060; font-size:16px";>
    Created by: &emsp;&emsp;&emsp; Mick Wiedermann<br>
    Student Number: &ensp;2220-58299<br>
    Email Address: &emsp;&ensp; mwiedermann@deakin.edu.au<br><br>
    <i>Computer Science Undergraduate - Data Science Major</i>
</p>

<hr style="height:1px; width:100%">


<p style="color:#336699; font-size:16px";><i>Importing the required modules</i></p>

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import sqlite3
import timeit

<hr style="height:1px; width:100%">

<h2 style="color:#336699"> Task 1: Downloading & Importing the Data</h2>

Downloading the following from NYC flights dataset:

* nycflights13_flights.csv – flights information,
* nycflights13_airlines.csv – decodes two letter carrier codes,
* nycflights13_airports.csv – airport data,
* nycflights13_planes.csv – plane data,
* nycflights13_weather.csv – hourly meteorological data for LGA, JFK, and EWR.

**Flights**

In [2]:
Flights = pd.read_csv('nycflights13_flights.csv', comment='#')
print(Flights.shape)
Flights.head(2) 

(336776, 19)


,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00


Columns:
* `dep_time` and `arr_time` - The actual depart and arrive times.
* `sched_dep_time` and `sched_arr_time` - The scheduled depart and arrive times.
* `dep_delay` and `arr_delay` - The depart and arrive delay time in minutes. 
* `carrier` - Airline code (name in airline dataset).
* `flight` and `tailnum` - ID numbers.
* `origin` and `dest` - Airport ID codes.
* `air_time` - Amount of time in the air in minutes.
* `distance` - Between origin and destination in miles. 
* `hour` and `minute` - Is the `sched_dep_time` broken into hour and minute. 
* `time_hour` - Schedule date and hour of departure (Can be used to join to flights dataset)

**Airlines**

In [3]:
Airlines = pd.read_csv('nycflights13_airlines.csv', comment='#')
print(Airlines.shape)
Airlines.head(2) 

(16, 2)


,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.


Columns for the "Airlines" dataset: 
* `carrier` is a 2 letter abbreviation of the airline name - `name`.

**Airports**

In [4]:
Airports = pd.read_csv('nycflights13_airports.csv', comment='#')
print(Airports.shape)
Airports.head(2)

(1458, 8)


,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago


Columns for the "Airports" dataset are as follows:
* `faa` - FAA Airport Code 
* `name`, `lat`, `lon`, - Self explanatory
* `alt` - Altitude of the airport
* `tz` - Time zone offset from GMT
* `dst` - Daylight Savings time zone (2nd Sunday of March - 1st Sunday November)
* `tzone` - IANA Time zone (as determined by GeoNames webservice)

**Planes**

In [5]:
Planes = pd.read_csv('nycflights13_planes.csv', comment='#')
print(Planes.shape)
Planes.head(2)

(3322, 9)


,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


Columns for the "Planes" dataset are fairly self explanitory with exception to `speed` which is the average crusing speed in mph and `year` which is the year of manufacture. 

**Weather**

In [6]:
Weather = pd.read_csv('nycflights13_weather.csv', comment='#')
print(Weather.shape)
Weather.head(2)

(26130, 15)


,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,0,37.04,21.92,53.97,230.0,10.35702,11.918651,0.0,1013.9,10.0,2013-01-01 01:00:00
1,EWR,2013,1,1,1,37.04,21.92,53.97,230.0,13.80936,15.891535,0.0,1013.0,10.0,2013-01-01 02:00:00


Columns for the "Weather" dataset:
* `origin` – weather station: LGA, JFK, or EWR,
* `year`, `month`, `day`, `hour` – time of recording,
* `temp`, `dewp` – temperature and dewpoint in degrees Fahrenheit,
* `humid` – relative humidity,
* `wind_dir`, `wind_speed`, `wind_gust` – wind direction (in degrees), speed and gust speed (in mph),
* `precip` – precipitation, in inches,
* `pressure` – sea level pressure in millibars,
* `visib` – visibility in miles,
* `time_hour` – date and hour (based on the year, month, day, hour fields) formatted as YYYY-mm-dd HH:MM:SS (actually, YYYY-mm-dd HH:00:00). However, due to a bug in the dataset, the data are shifted by 1 hour.

<hr style="height:1px; width:100%">

<h2 style="color:#336699"> Task 1.1: Establish SQLite Connection</h2>

Establishing a connection with a new SQLite database on the local disk.

In [7]:
import tempfile, os.path
dbfile = os.path.join(tempfile.mkdtemp(), "task_05D.db")
print(dbfile)

/var/folders/0n/s19df3pj3515r5_vrmr7rst80000gn/T/tmpt6u4kqai/task_05D.db


In [8]:
conn = sqlite3.connect(dbfile)

<hr style="height:1px; width:100%">

<h2 style="color:#336699"> Task 1.2: Exporting the CSV files</h2>

Exporting all the CSV files that have been imported into pandas to the new "Task_05D" SQLite database. 

In [9]:
Flights.to_sql("Flights", conn, index=False)
Airlines.to_sql("Airlines", conn, index=False)
Airports.to_sql("Airports", conn, index=False)
Planes.to_sql("Planes", conn, index=False)
Weather.to_sql("Weather", conn, index=False)

<hr style="height:1px; width:100%">

<h2 style="color:#336699"> Task 1.3: Writting Equivilant SQL Queries in Pandas</h2>

Writing code in pandas that yields equivalent results for each of the following SQL queries and explaining what each query does.

In [10]:
## Simple function to style timing results of function execution. 

def timer_results(start, end, obj):
    time = end - start
    display(Markdown(f'**Shape**: {obj.shape}. **Processing Time:** {time}.')) 
    
    return time

In [11]:
# Simple test function that generates a satisfying output when comparing objects.

def compare(obj_1, obj_2, t1, t2):
    if (t1 < t2):
        time = t2/t1
        display(Markdown(f'The **SQL** Query was a factor of {time:.2f} times faster than the Pandas.')) 
    else:
        time = t1/t2
        display(Markdown(f'The **Pandas** Query was a factor of {time:.2f} times faster than the SQL.')) 
        
    if (obj_1.equals(obj_2)): 
        display(Markdown('<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>')) 
    else:
        display(Markdown('<h3><span style="color:#cc3300"> :( Try Again </span><h3>'))

1. **`SELECT DISTINCT engine FROM planes`**

This query returns a list of distinct elements (i.e. no duplicates) from the `engine` column of the `planes` dataset. 

In [12]:
# SQL Query 
start = timeit.default_timer()

sql_1 = pd.read_sql_query('''
    SELECT DISTINCT engine FROM planes
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_1)

sql_1.head(2)

**Shape**: (6, 1). **Processing Time:** 0.0018843119999996105.

,engine
0,Turbo-fan
1,Turbo-jet


In [13]:
# Pandas Query
start = timeit.default_timer()

result = {'engine': Planes.engine.unique()} # Retreive unique engines 
pd_1 = pd.DataFrame(result)  # Convert to dataframe

end = timeit.default_timer()
t2 = timer_results(start, end, pd_1)

pd_1.head(2)

**Shape**: (6, 1). **Processing Time:** 0.0017793849999998557.

,engine
0,Turbo-fan
1,Turbo-jet


In [14]:
# Comparing the returned objects from each query above. 
compare(sql_1, pd_1, t1, t2)

pd.testing.assert_frame_equal(sql_1, pd_1)

The **Pandas** Query was a factor of 1.06 times faster than the SQL.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
2. **`SELECT DISTINCT type, engine FROM planes`**

This query is similar to the one above but combines the distinct values of two columns in this case being `type` and `engine` which does allow for dulicates but only so all distinct values in each column can be represented.

In [15]:
# SQL Query 
start = timeit.default_timer()

sql_2 = pd.read_sql_query('''
    SELECT DISTINCT type, engine FROM planes
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_2)

sql_2.head(2)

**Shape**: (7, 2). **Processing Time:** 0.001957712000000278.

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet


In [16]:
# Pandas Query 
start = timeit.default_timer()

pd_2 = Planes.groupby(['type', 'engine'], as_index=False, sort=False)['speed'].size() 
pd_2 = pd_2.drop('size', axis=1) 

end = timeit.default_timer()
t2 = timer_results(start, end, pd_2)

pd_2.head(2)

**Shape**: (7, 2). **Processing Time:** 0.00818661600000059.

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet


In [17]:
# Comparing the returned objects from each query above. 
compare(sql_2, pd_2, t1, t2)

pd.testing.assert_frame_equal(sql_2, pd_2)

The **SQL** Query was a factor of 4.18 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
3. **`SELECT COUNT(*), engine FROM planes 
GROUP BY engine`**

The query is requesting a count of the number of different engines from the planes dataset while displaying the counts by the engine type. 

In [18]:
start = timeit.default_timer()

sql_3 = pd.read_sql_query('''
    SELECT COUNT(*), engine FROM planes GROUP BY engine
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_3)

sql_3.head(2)

**Shape**: (6, 2). **Processing Time:** 0.002982926999999691.

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating


In [19]:
start = timeit.default_timer()

pd_3 = Planes.groupby(['engine'])['engine'].count() 
pd_3 = pd.DataFrame(pd_3) 
pd_3.rename(columns={'engine': 'COUNT(*)'}, inplace=True) 
pd_3 = pd_3.reset_index()  
pd_3 = pd_3.iloc[:,[1,0]] 

end = timeit.default_timer()
t2 = timer_results(start, end, pd_3)

pd_3.head()

**Shape**: (6, 2). **Processing Time:** 0.006466581000000637.

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop


In [20]:
# Comparing the returned objects from each query above. 
obj_1 = sql_3
obj_2 = pd_3

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 2.17 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
4. **`SELECT COUNT(*), engine, type FROM planes 
GROUP BY engine, type`**

This query asks for a count of all engine and type combinations grouped by engine and type. Essentially a count for each combination showing the combinations. 

In [21]:
start = timeit.default_timer()

sql_4 = pd.read_sql_query('''
    SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_4)

sql_4.head(2)

**Shape**: (7, 3). **Processing Time:** 0.0038785990000000936.

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine


In [22]:
start = timeit.default_timer()

pd_4 = Planes.groupby(['engine', 'type'], as_index=False)['manufacturer'].count()
pd_4 = pd_4.iloc[:, [2, 0, 1]]
pd_4.rename(columns={'manufacturer': 'COUNT(*)'}, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_4)

pd_4.head()

**Shape**: (7, 3). **Processing Time:** 0.010247674000000373.

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine


In [23]:
# Comparing the returned objects from each query above. 
obj_1 = sql_4
obj_2 = pd_4

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 2.64 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
5. **`SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes 
GROUP BY engine, manufacturer`**
      
In this query we are grouping all unique combinations of `engine` and `manufacturer` while taking the earliest, latest, and average `year` of manufacture. 

In [24]:
start = timeit.default_timer()

sql_5 = pd.read_sql_query('''
    SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes 
    GROUP BY engine, manufacturer''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_5)

sql_5.head(2)

**Shape**: (43, 5). **Processing Time:** 0.004651846999999876.

,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.0,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS


In [25]:
start = timeit.default_timer()

pd_5 = Planes.groupby(['engine', 'manufacturer'], as_index=False)
y_min = pd_5['year'].min()
y_min.rename(columns={'year' : 'MIN(year)'}, inplace=True)
y_max = pd_5['year'].max()
y_max.rename(columns={'year' : 'MAX(year)'}, inplace=True)
y_avg = pd_5['year'].mean()
y_avg.rename(columns={'year' : 'AVG(year)'}, inplace=True)

pd_5 = pd.merge(y_min, y_avg, how='left', on=['engine', 'manufacturer'])
pd_5 = pd.merge(pd_5, y_max, how='left', on=['engine', 'manufacturer'])
pd_5 = pd_5.iloc[:, [2, 3, 4, 0, 1]]

end = timeit.default_timer()
t2 = timer_results(start, end, pd_5)

pd_5.head()

**Shape**: (43, 5). **Processing Time:** 0.022955376000000527.

,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.0,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.0,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L


In [26]:
# Comparing the returned objects from each query above. 
obj_1 = sql_5
obj_2 = pd_5

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 4.93 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
6. **`SELECT * FROM planes WHERE speed IS NOT NULL`**

Here we are simply selecting everything from `Planes` that has a metric for `speed`.

In [27]:
start = timeit.default_timer()

sql_6 = pd.read_sql_query('''
    SELECT * FROM planes WHERE speed IS NOT NULL
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_6)

sql_6.head(2)

**Shape**: (23, 9). **Processing Time:** 0.0029626080000006993.

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating


In [28]:
start = timeit.default_timer()

pd_6 = Planes['speed'].dropna()
pd_6 = Planes.iloc[pd_6.index, :].reset_index(drop=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_6)

pd_6.head()

**Shape**: (23, 9). **Processing Time:** 0.001646970999999553.

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating


In [29]:
# Comparing the returned objects from each query above. 
obj_1 = sql_6
obj_2 = pd_6

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **Pandas** Query was a factor of 1.80 times faster than the SQL.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
7. **`SELECT tailnum FROM planes
WHERE seats BETWEEN 150 AND 190 AND year >= 2012`**

This query requests all `tailnum` from planes that have between 150 and 190 seats and that were manufactured in or after 2012.

In [30]:
start = timeit.default_timer()

sql_7 = pd.read_sql_query('''
    SELECT tailnum FROM planes
    WHERE seats BETWEEN 150 AND 190 AND year >= 2012
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_7)

sql_7.head(2)

**Shape**: (7, 1). **Processing Time:** 0.0028277040000004305.

,tailnum
0,N361VA
1,N849VA


In [31]:
start = timeit.default_timer()

pd_7 = Planes['tailnum'].loc[Planes['seats'] > 150].loc[Planes['seats'] < 190].loc[Planes['year'] >= 2012]
pd_7 = pd.DataFrame(pd_7).reset_index(drop=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_7)

pd_7.head()

**Shape**: (7, 1). **Processing Time:** 0.005386721999999899.

,tailnum
0,N361VA
1,N849VA
2,N851VA
3,N852VA
4,N853VA


In [32]:
# Comparing the returned objects from each query above. 
obj_1 = sql_7
obj_2 = pd_7

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 1.90 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
8. **`SELECT tailnum, manufacturer, seats FROM planes
WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390`**

Here the query wants the `tailnum`, `manufacturer`, and the number of `seats` for the select manufacturers being Boeing, Airbus, and Embraer that have more than 390 seats. 

In [33]:
start = timeit.default_timer()

sql_8 = pd.read_sql_query('''
    SELECT tailnum, manufacturer, seats FROM planes
    WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_8)

sql_8.head(2)

**Shape**: (13, 3). **Processing Time:** 0.0024597109999993094.

,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400


In [34]:
start = timeit.default_timer()

pd_8 = Planes.groupby(['tailnum', 'manufacturer', 'seats'], as_index=False)['speed'].size()
pd_8 = pd_8.loc[Planes['seats'] > 390].reset_index(drop=True)
pd_8.drop('size', axis=1, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_8)

pd_8.head()

**Shape**: (13, 3). **Processing Time:** 0.01015523099999971.

,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400
2,N272AT,BOEING,400
3,N57016,BOEING,400
4,N670US,BOEING,450


In [35]:
# Comparing the returned objects from each query above. 
obj_1 = sql_8
obj_2 = pd_8

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 4.13 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
9. **`SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC`**

The query returns the `year` and `seats` from `Planes` where the year of manufacture is at least 2012 ordering the years in ascending order and the seats in descending.

In [36]:
start = timeit.default_timer()

sql_9 = pd.read_sql_query('''
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012 ORDER BY year ASC, seats DESC
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_9)

sql_9.head(2)

**Shape**: (21, 2). **Processing Time:** 0.0016859440000001058.

,year,seats
0,2012.0,379
1,2012.0,377


In [37]:
start = timeit.default_timer()

pd_9 = Planes.groupby(['year', 'seats'], as_index=False)['speed'].count()
pd_9.drop('speed', axis=1, inplace=True)
pd_9 = pd_9.loc[pd_9['year'] >= 2012]
pd_9.sort_values(['year', 'seats'], ascending=[True, False], inplace=True)
pd_9.reset_index(drop=True, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_9)

pd_9.head()

**Shape**: (21, 2). **Processing Time:** 0.009182571000000195.

,year,seats
0,2012.0,379
1,2012.0,377
2,2012.0,260
3,2012.0,222
4,2012.0,200


In [38]:
# Comparing the returned objects from each query above. 
obj_1 = sql_9
obj_2 = pd_9

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 5.45 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
10. **`SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY seats DESC, year ASC`**

This query is the same as above returning the `year` and `seats` from `Planes` where the year of manufacture is at least 2012 but this time ordering the seats first in descending order and the years after in ascending.

In [39]:
start = timeit.default_timer()

sql_10 = pd.read_sql_query('''
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012 ORDER BY seats DESC, year ASC
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_10)

sql_10.head(2)

**Shape**: (21, 2). **Processing Time:** 0.002125465000000659.

,year,seats
0,2012.0,379
1,2013.0,379


In [40]:
start = timeit.default_timer()

pd_10 = Planes.groupby(['year', 'seats'], as_index=False)['speed'].count()
pd_10.drop('speed', axis=1, inplace=True)
pd_10 = pd_10.loc[pd_10['year'] >= 2012]
pd_10.sort_values(['seats', 'year'], ascending=[False, True], inplace=True)
pd_10.reset_index(drop=True, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_10)

pd_10.head()

**Shape**: (21, 2). **Processing Time:** 0.007464222000000298.

,year,seats
0,2012.0,379
1,2013.0,379
2,2012.0,377
3,2013.0,377
4,2012.0,260


In [41]:
# Comparing the returned objects from each query above. 
obj_1 = sql_10
obj_2 = pd_10

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 3.51 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
11. **`SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer`**

With this query we're looking for count of how many planes each `manufacturer` has in the dataset with more than 200 seats. 

In [42]:
start = timeit.default_timer()

sql_11 = pd.read_sql_query('''
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200 GROUP BY manufacturer
    ''', conn) 

end = timeit.default_timer()
t1 = timer_results(start, end, sql_11)

sql_11

**Shape**: (3, 2). **Processing Time:** 0.002174832999999765.

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [43]:
start = timeit.default_timer()

pd_11 = Planes.loc[Planes['seats'] > 200]
pd_11 = pd_11.groupby(['manufacturer'], as_index=False)['seats'].count() 
pd_11.rename(columns={'seats': 'COUNT(*)'}, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_11)

pd_11

**Shape**: (3, 2). **Processing Time:** 0.004900241000000527.

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [44]:
# Comparing the returned objects from each query above. 
obj_1 = sql_11
obj_2 = pd_11

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 2.25 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
12. **`SELECT manufacturer, COUNT(*) FROM planes
GROUP BY manufacturer HAVING COUNT(*) > 10`**

Here we're grouping by `manufacturer` while counting the number of planes in the dataset from each of the manufacturers.

In [45]:
start = timeit.default_timer()

sql_12 = pd.read_sql_query('''
    SELECT manufacturer, COUNT(*) FROM planes
    GROUP BY manufacturer HAVING COUNT(*) > 10
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_12)

sql_12.head(2)

**Shape**: (8, 2). **Processing Time:** 0.0033470189999995625.

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400


In [46]:
start = timeit.default_timer()

pd_12 = Planes.groupby(['manufacturer'])['manufacturer'].count()
pd_12 = pd.DataFrame(pd_12).rename(columns={'manufacturer': 'COUNT(*)'})
pd_12 = pd_12.loc[pd_12['COUNT(*)'] > 10]  
pd_12.reset_index(inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_12)

pd_12.head()

**Shape**: (8, 2). **Processing Time:** 0.004691290000000237.

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299


In [47]:
# Comparing the returned objects from each query above. 
obj_1 = sql_12
obj_2 = pd_12

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 1.40 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
13. **`SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10`**

Similar to the last query where we we're grouping by `manufacturer` while counting the number of planes in the dataset from each of the manufacturers but with the added parameter of the planes having more than 200 `seats`.

In [48]:
start = timeit.default_timer()

sql_13 = pd.read_sql_query('''
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_13)

sql_13

**Shape**: (2, 2). **Processing Time:** 0.0017881199999996156.

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [49]:
start = timeit.default_timer()

pd_13 = Planes.loc[Planes['seats'] > 200]
pd_13 = pd_13.groupby(['manufacturer', 'seats'])['manufacturer'].count()
pd_13 = pd.DataFrame(pd_13).rename(columns={'manufacturer': 'COUNT(*)'})
pd_13.reset_index(inplace=True)
pd_13 = pd_13.groupby(['manufacturer'], as_index=False)['COUNT(*)'].sum()
pd_13 = pd_13.loc[pd_13['COUNT(*)'] > 10]
pd_13.reset_index(drop=True, inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_13)

pd_13

**Shape**: (2, 2). **Processing Time:** 0.011369752999999427.

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [50]:
# Comparing the returned objects from each query above. 
obj_1 = sql_13
obj_2 = pd_13

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 6.36 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
14. **`SELECT manufacturer, COUNT(*) AS howmany
FROM planes
GROUP BY manufacturer
ORDER BY howmany DESC LIMIT 5`**

Here the query is requesting the number of planes in the dataset from each `manufacturer` sorted in descending order and limited to the top five results, the manufacturers with the most planes.  

In [51]:
start = timeit.default_timer()

sql_14 = pd.read_sql_query('''
    SELECT manufacturer, COUNT(*) AS howmany
    FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC LIMIT 5
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_14)

sql_14.head(2)

**Shape**: (5, 2). **Processing Time:** 0.0028992899999993327.

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400


In [52]:
start = timeit.default_timer()

pd_14 = Planes.groupby(['manufacturer'])['manufacturer'].count()
pd_14 = pd.DataFrame(pd_14)
pd_14.rename(columns={'manufacturer': 'howmany'}, inplace=True)
pd_14.sort_values('howmany', ascending=False, inplace=True)
pd_14 = pd_14[:5]
pd_14.reset_index(inplace=True)

end = timeit.default_timer()
t2 = timer_results(start, end, pd_14)

pd_14

**Shape**: (5, 2). **Processing Time:** 0.0050162079999998.

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299


In [53]:
# Comparing the returned objects from each query above. 
obj_1 = sql_14
obj_2 = pd_14

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **SQL** Query was a factor of 1.73 times faster than the Pandas.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
15. **`SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum`**

The query is perfomring a left join on the `Planes` dataset from the `Flights` dataset using the `tailnumber` as the unique identifier while limiting and renaming the columns from `Planes` to `plane_year`, `plane_speed`, and `plane_seats`. A left join here ensure all columns select from `Flights` remain even if there are no matches in `Planes`.

In [54]:
start = timeit.default_timer()

sql_15 = pd.read_sql_query('''
    SELECT
    flights.*,
    planes.year AS plane_year,
    planes.speed AS plane_speed,
    planes.seats AS plane_seats
    FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
    ''', conn)

end = timeit.default_timer()
t1 = timer_results(start, end, sql_15)

sql_15.head(2)

**Shape**: (336776, 22). **Processing Time:** 2.9499360810000006.

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,origin,dest,air_time,distance,hour,minute,time_hour,plane_year,plane_speed,plane_seats
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,1999.0,NaN,149.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,1998.0,NaN,149.0


In [55]:
start = timeit.default_timer()

pd_15 = Planes[['year', 'speed', 'seats', 'tailnum']]
pd_15 = pd_15.rename(columns={'year': 'plane_year', 'speed': 'plane_speed', 'seats': 'plane_seats'})
pd_15 = pd.merge(Flights, pd_15, how='left', on='tailnum')

end = timeit.default_timer()
t2 = timer_results(start, end, pd_15)

pd_15.head()

**Shape**: (336776, 22). **Processing Time:** 0.21031892199999902.

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,origin,dest,air_time,distance,hour,minute,time_hour,plane_year,plane_speed,plane_seats
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,1999.0,NaN,149.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,1998.0,NaN,149.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,1990.0,NaN,178.0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,2012.0,NaN,200.0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,1991.0,NaN,178.0


In [56]:
# Comparing the returned objects from each query above. 
obj_1 = sql_15
obj_2 = pd_15

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **Pandas** Query was a factor of 14.03 times faster than the SQL.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

----
16. **`SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier`**

This query combines attributes from three different datasets. Firstly grouping `carrier` and `tailnum` from flights so there are no unnecessary duplicates and naming the grouped object as `cartail`. Then performing a inner join (*an inner join only returns matches*) on `cartail` with `Planes` using `tailnum` as the identifier, before performing another inner join on `cartail` with `Airlines` using `carrier` as the unique identifier.

In [57]:
start = timeit.default_timer()

sql_16 = pd.read_sql_query('''
    SELECT planes.*, airlines.* FROM
    (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
    INNER JOIN planes ON cartail.tailnum=planes.tailnum
    INNER JOIN airlines ON cartail.carrier=airlines.carrier
    ''', conn)

# Sorting 'tailnum', 'carrier' values so the order doesn't cause issues. 
sql_16 = sql_16.sort_values(['tailnum', 'carrier']).reset_index(drop=True) 

end = timeit.default_timer()
t1 = timer_results(start, end, sql_16)

sql_16.head(2)

**Shape**: (3339, 11). **Processing Time:** 0.1989943400000005.

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.


In [58]:
start = timeit.default_timer()

cartail = Flights.groupby(['carrier', 'tailnum'], as_index=False)['distance'].sum()
cartail.drop('distance', axis=1, inplace=True)
pd_16 = pd.merge(Planes, cartail, how='inner', on='tailnum')
pd_16 = pd.merge(pd_16, Airlines, how='inner', on='carrier')

# Sorting 'tailnum', 'carrier' values to match the SQL output order.
pd_16 = pd_16.sort_values(['tailnum', 'carrier']).reset_index(drop=True) 

end = timeit.default_timer()
t2 = timer_results(start, end, pd_16)

pd_16.head()

**Shape**: (3339, 11). **Processing Time:** 0.078871165999999.

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.


In [59]:
# Comparing the returned objects from each query above. 
obj_1 = sql_16
obj_2 = pd_16

compare(obj_1, obj_2, t1, t2)
pd.testing.assert_frame_equal(obj_1, obj_2)

The **Pandas** Query was a factor of 2.52 times faster than the SQL.

<h3><span style="color:#008000">Pandas SQL & Pure Pandas objects are a match!</span><h3>

## Processing Speed of each Query

Looking over the results of the processing times from each of the queries, it seems that when the query is relatively simple SQL performs better, in most cases. I would assume that this is only due to the number of function calls required to wrangle the Pandas Dataframe into a similar object. If the Pandas query can be completed in one or two calls then Pandas seems to perform better. The more complex queries were faster with pure Pandas. 

<hr style="height:1px; width:100%">
<h3> References </h3>

* [Minimalist Data Wrangling with Python](https://datawranglingpy.gagolewski.com/index.html) - Dr habil. Marek Gagolewski. 
* [Pandas Documentation](https://pandas.pydata.org/docs/reference/frame.html) - Dataframes & Grouby Objects. 